In [ ]:
###BERT와 GPT
###BERT(Bidirectional Encoder Representations from Transformers) # 양방향으로..
###GPT(Generative Pre-trained Transformer) # 둘다 트랜스포머 but 좀 차이 있음.
###예: 문서 분류, 질의 응답(chatbot), 개체명인식(ner, 삼성 -> 기업이라고 인식), 문장생성.

###트랜스퍼 러닝(Transfer Learning): pre-trained, fine-tuning(미세조정)
###특정 용도에 맞춰 사전에 학습된(upstream 태스크) 모델을 다른 용도로 재사용(downstream 태스크)할 수 있는 기술.
###업스트림 태스크: 다음 단어 예측(GPT), 빈칸 채우기(BERT)->마스크드 언어 모델, 코퍼스의 문맥을 이해하는 문제->학습->per-trained(프리트레인드)모델(해결되면 pretraining이라고 함.)
###pre trained 사용하는 목적: 다운스트림을 하기위해, 재사용하여 응용하는데 있어 성능을 극대화.
###다운스트림 태스크: 문서 분류, 개체명 인식 등 구체적인 자연어 문제를 해결.
###ex) 독도는 우리 땅, 독도는 경상북도에 속한다, ..
###ex) 독도는 ___ 땅.
###eXtrainable AI(설명가능한 AI)
###자기지도학습: 데이터 내에서 X,Y데이터를 나누어 학습.
###파인튜닝: 사전학습모델의 가중치를 태스크에 맞춰 업데이트하는 기법.
###프롬프트튜닝(prompt tuning): 사전학습모델의 가중치를 태스크에 맞춰 모델 일부만 업데이트하는 기법.
###In-context 러닝: 다운스트림 테스크 데이터의 일부만 사용, 모델을 업데이트 하지 않음.
###퓨샷(few-shot)러닝(다운스트림 테스크 데이터 몇 건만 사용.).
###제로샷러닝(zero-shot)(다운스트림 테스크 데이터 전혀 사용 안함).
###원샷러닝(다운스트림 테스크 데이터 11건만 사용, 하나밖에 없는데 학습함).

###토큰화: 문자, 단어, 문장, 서브단어
###바이트 쌍 인코딩: 최근 모델 토큰화 기법, GPT
###단어 조각 인코딩(워드피스): BERT


###바이트 쌍 인코딩
###정보를 압축하기 위해 사용됬던 알고리즘(문자를 압축함.)
###연속하는 문자를 통으로 압축
###예) 'aaabdaabb' (a,b,c) 사전에는 3가지 단어가 있음.
###위를 xabcxbb로 압축

###
###OOV: 단어가 사전에 없는 경우
###단어집합 - low:5, lower:2, newest:6, widest:3=> l,o,w,e,r,n,s,t,i,d
###ex) lowest 입력->OOV

###워드피스: 바이트 쌍 인코딩과 비슷, 코퍼스에서 자주 등장한 문자열을 토큰화.
###차이점은 문자열을 합치는 기준이 다름. 바이트 쌍 인코딩에서는 빈도를 기준으로 합치기
###워드피스는 우도(likehood)가 가장 높은 쌍을 합치기.

###코퍼스에 a, b문자에 대해 계산 방법(n: 전체 글자 수, #ab:ab문자열의 빈도수,
### #a:a문자 빈두소, #b:b문자 빈도수)
###=(#ab/n) / ((#a/n)*(#b/n)) 결과값이 큰 값에 해당되는 쌍부터 합치기.

###센턴스피스: 문장에서 단어를 분리하는 패키지

###허깅페이스 토크나이저: 자주 등장하는 서브 워드들을 하나의 토큰으로 토큰화

In [ ]:
###seq2seq바탕으로 토큰화
###se2seq: 챗봇, 번역기, 내용요약, 음성=>텍스트 or 이미지=>CNN=>상황분
###context vector:
###문장을 구성하는 순서 정보, 단어들에 대한 정보를 바탕으로 하나로 압축되어 있는 벡터.
###인코더에 대한 모든 정보가 들어있음. 너무 길어지면 한계가 있음(크기를 잘 정의해야함.)
###인코더에 LSTM이 들어가 있음.
###디코더에서 단어들을 예측, 다음으로 eos에서 끝남.(eos: 문장 생성 시작과 종료를 알리는 신호.)

###번역기 생성

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
# ,가 아닌 다른 문자로 되었는 경우, sep옵션에서 기술.
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/kor-eng/kor.txt", sep='\t',
            names=['src', 'tar', 'lic'])
df

In [ ]:
del df['lic']

In [ ]:
df

In [ ]:
len(df)

5870

In [ ]:
# 탭문자로 구분되기 때문에 데이터를 더 쉽게 처리하고 분할할 수 있음.
df.tar=df.tar.apply(lambda x: '\t'+x+'\t') # sos:\t, eos:\n x에는 문장하나하나가 들어옴.
df

In [ ]:
srcVocab=set()
for line in df.src:
  for c in line: # c에는 문자열을 구성하는 부분, 문자단위로 읽어짐.
    srcVocab.add(c)

In [ ]:
tarVocab=set()
for line in df.tar:
  for c in line:
    tarVocab.add(c)

In [ ]:
len(tarVocab) # 1009
len(srcVocab) # 74

74

In [ ]:
# 번호부여.
srcVocab=sorted(list(srcVocab))


tarVocab=sorted(list(tarVocab))

In [ ]:
# 나중에 문자의 개수를 알아야함 따라서 개수 저장한다.
srcVocabSize=len(srcVocab)+1
tarVocabSize=len(tarVocab)+1 #0번을 사용하지 않음.

In [ ]:
tarVocab # 만약 1번부터 사용하고 싶다면 0번을 버려야함. +1을 해줌.

###문자 단위로 나누어진 단어에 숫자부여 작업

In [ ]:
# 0부터 시작하기 때문에 1을 더해줌.  리스트를 만들고 딕에 넣으면 딕셔너리가 됨.
tar_to_index=dict([(w, i+1) for i, w in enumerate(tarVocab)])

In [ ]:
tar_to_index

In [ ]:
# src 데이터도 동일한 수행

In [ ]:
src_to_index=dict([(w, i+1) for i, w in enumerate(srcVocab)])

###입력 문장에 대해서 정수 인코딩

In [ ]:
enc_input=[]
for line in df.src:
  enc_line=[]
  for c in line:
    enc_line.append(src_to_index[c])
  enc_input.append(enc_line)

In [ ]:
print(enc_input)

In [ ]:
dec_input=[]
for line in df.tar:
  dec_line=[]
  for c in line:
    dec_line.append(tar_to_index[c])
  dec_input.append(dec_line)

In [ ]:
print(dec_line)

[1, 696, 579, 696, 2, 643, 760, 2, 635, 697, 2, 543, 525, 640, 233, 2, 628, 310, 2, 183, 710, 697, 291, 2, 729, 981, 1006, 2, 302, 2, 609, 395, 233, 2, 643, 710, 662, 2, 81, 976, 944, 65, 177, 2, 118, 2, 446, 256, 696, 2, 525, 984, 697, 2, 738, 719, 944, 760, 10, 2, 118, 343, 264, 2, 699, 49, 697, 2, 558, 455, 2, 415, 696, 2, 518, 329, 387, 2, 609, 86, 2, 760, 190, 233, 2, 518, 697, 47, 2, 277, 2, 126, 985, 374, 2, 57, 233, 244, 86, 2, 530, 48, 951, 2, 472, 414, 8, 2, 314, 2, 118, 2, 558, 455, 2, 415, 2, 751, 2, 646, 643, 2, 415, 2, 783, 2, 697, 944, 387, 2, 714, 2, 609, 2, 558, 2, 706, 86, 8, 2, 118, 379, 86, 2, 177, 534, 2, 118, 2, 646, 643, 2, 415, 2, 751, 640, 2, 946, 279, 2, 415, 387, 2, 831, 102, 47, 2, 277, 2, 558, 2, 706, 244, 414, 8, 2, 118, 379, 86, 2, 314, 2, 387, 700, 2, 671, 379, 47, 2, 697, 2, 543, 525, 640, 2, 521, 86, 2, 706, 233, 2, 558, 455, 387, 2, 415, 696, 2, 518, 329, 293, 387, 2, 126, 629, 944, 86, 2, 706, 244, 414, 8, 2, 302, 2, 395, 233, 2, 183, 710, 233, 2, 76

In [ ]:
# 앞에 1은 필요없기때문에 빼는작업.
dec_tar=[]
for line in df.tar: # df.tar에는 한국어문장이 들어가 있음.
  enc_line=[]
  t=0
  for c in line: # <sos>가 들어가 있음. 훈련할때는 sos가 필요, 테스트할때는 <sos>가 필요없음. 그래서 첫번째는 뺌.
    if t>0:
      enc_line.append(tar_to_index[c])
    t=t+1
  dec_tar.append(enc_line)

In [ ]:
dec_tar

In [ ]:
# 한국어와 영어문장의 최대길이를 조사함.
max_src_len=max([len(line) for line in df.src]) # 537, 영어문장 들어가 있음. 모든 입력 문장의 최대길이를 구함.
max_tar_len=max([len(line) for line in df.tar]) # 298, 한국어문장

In [ ]:
# 영어는 영어대로 한국어는 한국어대로 패딩작업을 한다.
enc_input=pad_sequences(enc_input, maxlen=max_src_len, padding='pre')
dec_input=pad_sequences(dec_input, maxlen=max_tar_len, padding='pre')
dec_tar=pad_sequences(dec_tar, maxlen=max_tar_len, padding='pre')

In [ ]:
# 원핫인코딩 작업.
enc_input=to_categorical(enc_input)
dec_input=to_categorical(dec_input)
dec_tar=to_categorical(dec_tar)

In [ ]:
enc_input.shape # (3000, 26, 68) LSTM셀이 26개,
dec_input.shape # (3000, 30, 805) 30개로 잡아둔것(출력이 30개 나오는 것은 아님.)
dec_input.shape # (3000, 30,805) 805: 단어에 대한 차원, 종류가 한글의 경우에는 805가 있음.

(5870, 298)

### 번역을 하는 작업

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, Input
from tensorflow.keras.models import Model
import numpy as np

##모델 생성에는 두가지 방식이있음.
###model을 만들고 레이어를 추가하는 방식.
###model=Sequential()
###model.add(Dense(10, input_dim=100, activation=relu) # 입력계층의 대한 seq객체.
###다음층을 추가할때는 저절로 10으로 됨. 출력만 설정하면됨.
###model.add(Dense(5,))

### functional API
###input(shape=(None, srcVocabSize)) # srcVocabsize: 입력 문장의 개수는 정해져 있지 않음.
###차원은 단어의 개수, 입력 데이터의 정해져있음.

###인코더

In [ ]:
# None: 트레이닝할때 들어가는 입력의 개수가 계속해서 변화함. 그래서 일일이 숫자를 바꾸는 것은 번거로움 그래서 None으로 하면 알아서 해줌.
# srcVocabSize: 차원의 정보.
enc_inputs=input(shape=(None, srcVocabSize))

#True: 모든셀들이 출력, False: 마지막셀만 출력
# 상태(마지막 LSTM셀에서의 디코더쪽으로 전달.)를 리턴할 것이냐.
# units=256: LSTM출력차원의 수
# stateH: 은닉 상태 정보, stateC: 셀 상태 정보.
enc_lstm=LSTM(units=256, return_state=True)
enc_outputs, stateH, stateC=enc_lstm(enc_inputs)
enc_states=[stateH, stateC] # context vector가 된다. 전체문장에 대한 정보가 담겨있음.
                            # 은닉과 셀 상태의 정보, 마지막 셀에서의 출력.

TypeError: ignored

In [ ]:
###디코더
dec_inputs=Input(shape=(None , tarVocabSize))
dec_lstm=LSTM(units=256, return_state=True, return_sequences=True)

# 인코더 => 정보 => 디코더
dec_outputs, _, _ =dec_lstm(dec_inputs, initial_state=enc_states)
dec_layer=Dense(tarVocabSize, activation='softmax')
dec_outputs=dec_layer(dec_outputs)

### 모델의 만드는 작업

In [ ]:
model=Model([enc_inputs, dec_inputs], dec_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_corssentropy')

In [ ]:
model.fit(x=[enc_input, dec_input], y=dec_tar, batch_size=64, epochs=10,
          validation_split=0.2)

In [ ]:
enc_model=Model(inputs=enc_inputs, outputs=enc_states)

In [ ]:
#상태 정보를 저장하는 변수
decoder_state_input_h = input(shape=(256,))
decoder_state_input_c = input(shape=(256,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

In [ ]:
dec_outputs, state_h, state_c=dec_lstm(dec_inputs, initial_state=decoder_states_inputs)

In [ ]:
decoder_state=[state_h, state_c]
dec_outputs=dec_layer(dec_outputs) # softmax
Model(inputs=[dec_inputs]+decoder_states_inputs, outputs=[dec_outputs]+decoder_states)

In [ ]:
def dec_sequence(input_seq):

In [ ]:
input_seq=enc_input[3] #입력문장 Run.
dec_sentence=dec_sequence(input_seq)